In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer
import pickle
from asif import ASIF, extract_candidate_sets_from_clusters
from operator import itemgetter

/home/luigi/anaconda3/envs/chocolm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.auto.tokenization_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

In [7]:
data_folder = "data"

print("Loading text encoder...")
model = SentenceTransformer('xlm-roberta-base')

print("Loading chord encorder")
chord_encoder = SentenceTransformer('jammai/chocolm-modernbert-base')

print("Loading dataset...")
lyrics = pickle.load(open(f"{data_folder}/lyrics.pkl", "rb"))
chords = pickle.load(open(f"{data_folder}/chords.pkl", "rb"))
artist_song = pickle.load(open(f"{data_folder}/artist_song.pkl", "rb"))

print("Loading precomputed embeddings...")
chords_embeddings = pickle.load(open(f"{data_folder}/chords_embeddings_sbert_chocolm.pkl", "rb"))
lyrics_embeddings = pickle.load(open(f"{data_folder}/lyrics_embeddings_sbert_roberta.pkl", "rb"))

kmeans_lyrics = pickle.load(open(f"{data_folder}/lyrics_kmeans.pkl", "rb"))
kmeans_chords = pickle.load(open(f"{data_folder}/chords_kmeans.pkl", "rb"))

lyrics_candidates = extract_candidate_sets_from_clusters(kmeans_lyrics, lyrics, retrieve_ids=True)
chords_candidates = extract_candidate_sets_from_clusters(kmeans_chords, chords, retrieve_ids=True)

Loading text encoder...


NameError: name 'SentenceTransformer' is not defined

In [5]:
print("Initialising asif...")
asif = ASIF(
    lyrics_candidates,
    chords_candidates,
    torch.from_numpy(kmeans_lyrics.cluster_centers_),
    torch.from_numpy(kmeans_chords.cluster_centers_),
    lyrics_embeddings,
    chords_embeddings
)

Initialising asif...


In [6]:
text_embedding = model.encode(["I'm waking up from ash and dust"])

In [7]:
text_embedding = torch.from_numpy(text_embedding)

In [8]:
lyrics_relative_coordinates = asif.compute_relative_coordinates_vs_space1(text_embedding)

In [9]:
pred_candidate = (1 / (1 + torch.cdist(lyrics_relative_coordinates, asif.candidate_embeddings2_rc))).argmax(dim=1)

In [10]:
pred_candidate

tensor([40])

In [11]:
candidates_ids = itemgetter(*pred_candidate.tolist())(chords_candidates)

In [12]:
candidates_ids[0]

262146

In [13]:
lyrics[candidates_ids[0]], chords[candidates_ids[0]], artist_song[candidates_ids[0]]

('Ai, ai, ai, ai ',
 'A:min G:maj F:maj E:maj',
 {'artist': 'Anitta', 'song': 'Pantera'})

In [14]:
itemgetter(*candidates_ids)(chords), itemgetter(*candidates_ids)(lyrics), itemgetter(*candidates_ids)(artist_song)

(('A:min G:maj F:maj E:maj',
  'C:maj F:maj D:min G:min',
  'G:maj A:min C:maj',
  'G:maj A:min G:maj C:maj G:maj A:min G:maj',
  'A:min G:maj F:maj E:maj',
  'G:maj A:min C:maj',
  'A:min G:maj F:maj E:maj',
  'G:maj E:min D:maj C:maj B:min A:min G:maj C:maj G:maj A:min G:maj',
  'G:maj E:min D:maj C:maj B:min A:min G:maj C:maj G:maj A:min G:maj',
  'C:maj B:min A:min G:maj F:maj C:maj',
  'C:maj B:min A:min G:maj F:maj C:maj',
  'C:maj B:min A:min G:maj F:maj C:maj',
  'E:min C:maj G:maj/5',
  'C:maj G:maj A:min G:maj',
  'A:min G:maj F:maj C:maj',
  'A:min G:maj F:maj G:maj',
  'D:maj G:maj C:maj A:min',
  'D:maj G:maj C:maj A:min',
  'G:maj A:min C:maj',
  'G:maj A:min C:maj',
  'D:maj G:maj C:maj A:min',
  'E:min G:maj F:maj C:maj',
  'A:min G:maj F:maj',
  'A:min G:maj C:maj',
  'A:min G:maj F:maj',
  'A:min G:maj F:maj',
  'A:min F:maj7 G:maj',
  'A:min G:maj F:maj',
  'A:min G:maj F:maj',
  'A:min G:maj F:maj',
  'C:maj G:maj A:min G:maj F:maj7',
  'A:min F:maj7 G:maj',
  'C:ma

In [15]:
resulting_chords = itemgetter(*candidates_ids)(chords)
resulting_lyrics = itemgetter(*candidates_ids)(lyrics)
resulting_artist = map(lambda x : x["artist"], itemgetter(*candidates_ids)(artist_song))
resulting_song = map(lambda x : x["song"], itemgetter(*candidates_ids)(artist_song))


In [ ]:
len(candidates_ids)

91379

In [16]:
import pandas as pd

df = pd.DataFrame({"chords": resulting_chords, "lyrics": resulting_lyrics, "artist": resulting_artist, "song": resulting_song })

len(df)

91379

In [17]:
df = df.drop_duplicates()
len(df)

68852

In [18]:
df = df[~df["chords"].str.contains("N")]

In [19]:
len(df)

66340

In [20]:
def ffff(x):
    print(type(x))
    print(x)
    print(len(x))

aa = df[df["chords"].str.contains("A:min G:maj")].groupby(by="chords")

In [23]:
aa.groups

{'A:maj A:min G:maj C:maj': [12279], 'A:maj D:maj G:maj C:maj A:min G:maj A:min G:maj': [71919], 'A:maj F:maj G:maj C:maj F:maj A:min G:maj C:maj F:maj A:min G:maj': [70131], 'A:min A:min G:maj C:maj C:maj G#:maj': [70549], 'A:min A:min G:maj F:maj': [17443, 20960, 20966, 20970, 36261, 59821, 70609, 74524, 77452], 'A:min A:min G:maj F:maj E:7': [50967], 'A:min A:min G:maj F:maj F:maj C:maj': [74879], 'A:min A:min G:maj G:maj E:maj E:maj': [15363], 'A:min A:min G:maj/3 C:maj': [69318], 'A:min A:min G:maj/3 C:maj D:maj': [60242, 60243], 'A:min A:min(9)/9 C:maj A:min G:maj C:maj A:min G:maj C:maj': [39749, 39750, 39751, 39755, 39756], 'A:min A:min/5 A:min G:maj C:maj D:maj': [17341, 17343, 17345], 'A:min B:maj C:maj A:min C:maj G:maj A:min C:maj G:maj A:min G:maj A:min': [19026], 'A:min C:(*3,*5) A:min G:maj': [24383], 'A:min C:maj A:min E:min C:maj A:min G:maj/3 C:maj D:maj': [24015], 'A:min C:maj A:min G:maj': [1548, 2676, 18472, 22391, 22392, 25714, 26249, 26251, 30552, 57535, 61487, 7

In [28]:
aa.groups["A:maj A:min G:maj C:maj"]

Index([12279], dtype='int64')

In [60]:
df.loc[[0,1]][["lyrics", "artist", "song"]].values.tolist()

[['Ai, ai, ai, ai ', 'Anitta', 'Pantera'],
 ['Por te querer tanto mais que a mim ', 'Reginaldo Rossi', 'Jeito De Santa']]

In [61]:
{k:df.loc[v][["lyrics", "artist", "song"]].values.tolist() for (k,v) in aa.groups.items()}

{'A:maj A:min G:maj C:maj': [["And we don't let go when we're alone when I'm holding her so tight ",
   'Bon Jovi',
   'Always Run To You']],
 'A:maj D:maj G:maj C:maj A:min G:maj A:min G:maj': [['No no, no no??. ',
   'Josh Garrels',
   'Cynicism']],
 'A:maj F:maj G:maj C:maj F:maj A:min G:maj C:maj F:maj A:min G:maj': [["There's nothing like the view from the cheap seats  ",
   'Alabama',
   'Cheap Seats']],
 'A:min A:min G:maj C:maj C:maj G#:maj': [[' Thanks to kismet, kismet, kismet (KEY CHANGE HERE) ',
   'Elvis Presley',
   'Kismet']],
 'A:min A:min G:maj F:maj': [['passe to la faim ',
   'King',
   'La Maison De Dieu'],
  ['Inventei vitórias', 'Engenheiros Do Hawaii', 'A Conquista do Espelho'],
  ['Numa sala de torturas', 'Engenheiros Do Hawaii', 'A Conquista do Espelho'],
  ['Um país em maus lençóis',
   'Engenheiros Do Hawaii',
   'A Conquista do Espelho'],
  ['words a hundred times ', 'Tracy Lawrence', 'Is that a tear i hear'],
  ['laß                      die Sonne putzen! '

In [62]:
grouped = df.groupby(by="chords")
{k:df.loc[v][["lyrics", "artist", "song"]].values.tolist() for (k,v) in grouped.groups.items()}

{'A#:maj A:min G:min C:maj F:maj': [['es la boca que sabe de los besos que saben mal. ',
   'Sabroso',
   'Boca']],
 'A#:maj C:maj A:min D:7 G:min C:maj A:(*3,*5) A:maj': [[' Espero que tu siempre creas en mi tal como yo e creido en ti ',
   'José José',
   'Te Quiero Tal Como Eres']],
 'A#:maj C:maj A:min D:min G:maj': [['Ya no la molestaré,   no me  puede querer  ',
   'Juan Gabriel',
   'Solo Se Que Fue En Marzo']],
 'A#:maj C:maj D:min F:maj C:maj G:maj D:min': [['     Lean in to waaaalk ',
   'John Frusciante',
   'A Doubt']],
 'A#:maj C:maj D:min F:maj C:maj G:min D:min A#:maj C:maj': [['     And all that I need is a doubt ',
   'John Frusciante',
   'A Doubt']],
 'A#:maj C:maj D:min G:min': [['O Senhor me salvou, me curou, me ungiu ',
   'Kleber Lucas',
   'Profeta Da Esperança']],
 'A#:maj C:maj F:maj G:min C:maj': [['y no puedo imaginarme este mundo sin tí. ',
   'Sabroso',
   'Eres Especial']],
 'A#:maj C:maj G:maj D:min': [['     We dreamed up toniiiiight ',
   'John Fruscia

In [62]:
df.values.tolist()

[['A:min G:maj F:maj E:maj', 'Ai, ai, ai, ai ', 'Anitta', 'Pantera'],
 ['C:maj F:maj D:min G:min',
  'Por te querer tanto mais que a mim ',
  'Reginaldo Rossi',
  'Jeito De Santa'],
 ['G:maj A:min C:maj',
  "It's our last time to say goodnight ",
  'The Maine',
  "We'll All Be"],
 ['G:maj A:min G:maj C:maj G:maj A:min G:maj',
  'To lay     me  down. ',
  'Jerry Garcia',
  'To Lay Me Down'],
 ['G:maj A:min C:maj',
  "we'll see you around, and we'll sing it again, ",
  'The Maine',
  "We'll All Be"],
 ['G:maj E:min D:maj C:maj B:min A:min G:maj C:maj G:maj A:min G:maj',
  'To be with you,    once more,  To be  with you  ',
  'Jerry Garcia',
  'To Lay Me Down'],
 ['G:maj E:min D:maj C:maj B:min A:min G:maj C:maj G:maj A:min G:maj',
  'To lie with you,    once more,  To lie    with you  ',
  'Jerry Garcia',
  'To Lay Me Down'],
 ['C:maj B:min A:min G:maj F:maj C:maj',
  'to lay   me    down, one  last time  ',
  'Jerry Garcia',
  'To Lay Me Down'],
 ['C:maj B:min A:min G:maj F:maj C:maj',


In [64]:
text_embedding.size()

torch.Size([1, 768])

In [72]:
lyrics_embeddings.size()

torch.Size([3517221, 768])

In [ ]:
lyrics_embeddings[candidates_ids].size

torch.Size([91379, 768])

In [71]:
asif.candidate_embeddings2_rc.size()

torch.Size([64, 3517221])

In [1]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [4]:
lyrics_embeddings.size()[2]

NameError: name 'lyrics_embeddings' is not defined

In [ ]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(lyrics_embeddings.size()[2])   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [3]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751738 7.20763   7.2511625]
 [0.        6.3235645 6.684581  6.799946 ]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279875 7.662846 ]
 [0.        6.7638035 7.2951202 7.3688145]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]
